In [1]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M
from scipy.optimize import fsolve

In [2]:
############ Initilisation #################
############################################

#### Constants #####
L_x = 1e6          # [m] zonal extent 
L_y = 2e6          # [m] meridional extent 
A = L_x*L_y        # [m^2] surface area of interior 
P = 2*L_x+2*L_y    # [m] Perimeter 

g = 9.81           # [m/s^2] gravitational acceleration 
C_p = 3.994e3        # [J/(kg K)] specific heat capacity of sea water 
c = 0.006          # [] bottom slope coefficient
s = 0.02           # [] average bottom slope around the perimeter of the marginal sea

#### Parameters ####
H = 1000           # [m] sill height 
f_0 = 1.2e-4       # [s^-1] Coriolis parameter 
Gamma = 20         # [W/(m^2 C)] Restoring strength 
alpha_T = 0.2      # [kg m^-3 C^-1] thermal expansion coefficient; source: Ypma[2020]
alpha_S = 0.8      # [kg m^-3] Haline expansion coefficient 
#T_1 = 8.87         # [Celsius] ingoing temperature 
T_A = 2.8          # [Celsius] Atmospheric temperature 
ro = 1028          # [kg m^-3] density sea water 
S = 35             # [ppt] net evaporation minus precipitation
#S_1 = 35.2         # [g kg^-1] incoming salinity; source Lambert                          ! is ppt = g kg^-1 ???
E = -2e-8          # [m/s] a representative ocean salinity; source: Spall[2015]

#### functions ####
L = H/s           # [m] width of sloping topography of the boundary current 
#L = 75e3

In [7]:
steps=7          # test values
dx=10            # test values

T_0 = np.zeros(steps)
S_0 = np.zeros(steps)
V = np.zeros(steps)
T_1 = np.zeros(steps)
S_1 = np.zeros(steps)

T_1[0]=8.87 
S_1[0]=35.2 

for i in range(steps):
    def main(x): # function with seven unkowns:  T_0,  S_0,    V, T_1+1, S_1+1,  V_+1 
                 #                              x[0], x[1], x[2],  x[3],  x[3],  x[5]
        return [c*x[2]*(T_1[i]-x[0])-A*Gamma*(x[0]-T_A)/(ro*C_p),
                c*x[2]*(T_1[i]-x[0])-A*E*S,
                x[3]-x[2]*(T_1[i]-x[0])*c*dx/(H*L)-(2*x[2]*T_1[i]-T_1[i]*x[5])/x[2],
                x[5]-x[2]*(T_1[i]-x[0])*c*dx/(H*L)-(2*x[2]*T_1[i]-x[2]*x[3])/T_1[i],
                x[3]-x[2]*(S_1[i]-x[1])*c*dx/(H*L)-(2*x[2]*S_1[i]-S_1[i]*x[5])/x[2],
                x[5]-x[2]*(S_1[i]-x[1])*c*dx/(H*L)-(2*x[2]*S_1[i]-x[2]*x[3])/S_1[i]]

    [T_0[i],S_0[i],V[i],T_1[i+1],S_1[i+1],V[i+1]] = fsolve(main, [1,1,1,1,1]) 
    # V[i] and V[i+1] are calculated at the same time - that's not so good I think

IndexError: index 5 is out of bounds for axis 0 with size 5